# Attempting to write an octree in python

I want this to work with VERY large data sets that can't be stored fully in memory.  So my procedure will be as follows:
- need to read in line-by-line and clear memory every X MB (or maybe every X particles;can I check memory load in python?)
- go down to nodes with containing N particles
- need to write out tree with node sizes and centers and also ending nodes with actual particles

I will work in the octreeStream.py file.  This is working, though once it has to start reading and writing lots of files, it becomes VERY slow (setting NMemoryMax to be as large as possible will speed things up).

The ultimate goal is to use this within Firefly.  First I create the python code.  Then I will create a simple WebGL utility to use it (hopefully showing more Gaia data than it can handle w/o the octree).  Then we will work on incorporating this into Firefly.

In [ ]:
%load_ext autoreload
%autoreload 2

from octreeStream import octreeStream

In [ ]:
#I'll use a Gaia data set.  I've already compiled this one from DR2.  For production, I'd want to recompile from EDR3
# WORK/Gaia/GaiaCompiled_RV_TAP.txt
# for some reason this is a space-separated and not comma-separated, but here are the colummns
# 'x', 'y', 'z', 'vx', 'vy', 'vz', 'teff_val', 'phot_g_mean_mag', 'flag'
# I believe flag == 1 if there is no RV; for now all I care about is the x,y,z, position

o = octreeStream('/Users/ageller/WORK/Gaia/GaiaCompiled_RV_TAP.txt', header = 0, delim = ' ', 
                 NNodeMax = 10000, NMemoryMax = 1e6, lineNmax=2e6, verbose=1, 
                 path='/Users/ageller/VISUALIZATIONS/octree_threejs_Gaia/WebGL_octreePartition/src/data/octreeNodes')

In [ ]:
o.compileOctree()

In [ ]:
o.checkNodeFiles()

In [ ]:
o.populateAllNodesFromFiles()

In [ ]:
n = 0
count = 0
for i in o.baseNodeIndices:
    if (o.nodes[i]['Nparticles'] > 0):
        #print(i, o.nodes[i]['id'], o.nodes[i]['Nparticles'], len(o.nodes[i]['particles']), o.nodes[i]['x'],o.nodes[i]['xWidth'])
        n += o.nodes[i]['Nparticles']
        count += 1
print(n, count)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

%matplotlib notebook

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(projection='3d')
ax.set_xlim(-200,200)
ax.set_ylim(-200,200)
ax.set_zlim(-200,200)

n = 0
for i in o.baseNodeIndices:
    if (o.nodes[i]['Nparticles'] > 0 ):
        parts = np.array(o.nodes[i]['particles'])
        x = parts[:,0]
        y = parts[:,1]
        z = parts[:,2]
        ax.scatter(x,y, z)


In [ ]:
#I'd like to check this in Firefly
# - I will try to recreate my previous workflow with per-particle colors with the current version of python